In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle, islice
import datetime as datetime
import glob
from matplotlib import pyplot
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, make_scorer
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
import warnings

%matplotlib inline

plt.style.use('default')
sns.set(style="whitegrid")
pd.options.display.float_format = '{:20,.2f}'.format
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (15, 5)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from catboost import CatBoostRegressor

In [2]:
# for preprocessing the data
from sklearn.preprocessing import StandardScaler
# the model
from sklearn.ensemble import RandomForestClassifier
# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline
# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [3]:
train_labels = pd.read_csv('training_train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id')
train_values = pd.read_csv('training_train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

In [4]:
train_values.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [5]:
train = train_values.merge(train_labels, on='building_id')
train

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007913,10,1382,11935,2,60,11,6,t,u,x,...,0,0,0,0,0,0,0,0,0,2
349181,13,1154,1044,2,0,8,4,n,r,n,...,0,0,0,0,0,0,0,0,0,1
1009561,26,1001,11008,1,5,15,3,t,i,x,...,0,0,0,0,0,0,0,0,0,2


In [6]:
train = train.drop('geo_level_1_id', axis=1)
train = train.drop('geo_level_2_id', axis=1)
train = train.drop('geo_level_3_id', axis=1)
train = train.drop('land_surface_condition', axis=1)
#train = train.drop('count_floors_pre_eq', axis=1)

In [7]:
#edad promedio segun cantidad de pisos
train['edad_promedio'] = train.groupby('count_floors_pre_eq')['age'].transform('mean')

In [8]:
train.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,...,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,r,n,f,q,t,d,...,0,0,0,0,0,0,0,0,3,25.82
28830,2,10,8,7,r,n,x,q,s,d,...,0,0,0,0,0,0,0,0,2,25.82
94947,2,10,5,5,r,n,f,x,t,d,...,0,0,0,0,0,0,0,0,3,25.82
590882,2,10,6,5,r,n,f,x,s,d,...,0,0,0,0,0,0,0,0,2,25.82
201944,3,30,8,9,r,n,f,x,s,d,...,0,0,0,0,0,0,0,0,3,30.18


In [9]:
#cantidad de tipos de techo segun edad
train['tipo_techo'] = train.groupby('age')['roof_type'].transform('count')
#train = train.drop('roof_type', axis=1)

In [10]:
train.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio,tipo_techo
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,r,n,f,q,t,d,...,0,0,0,0,0,0,0,3,25.82,9299
28830,2,10,8,7,r,n,x,q,s,d,...,0,0,0,0,0,0,0,2,25.82,19889
94947,2,10,5,5,r,n,f,x,t,d,...,0,0,0,0,0,0,0,3,25.82,19889
590882,2,10,6,5,r,n,f,x,s,d,...,0,0,0,0,0,0,0,2,25.82,19889
201944,3,30,8,9,r,n,f,x,s,d,...,0,0,0,0,0,0,0,3,30.18,9299


In [11]:
#cantidad de familias por edificacion
train['flias_afectadas'] = train.groupby('building_id')['count_families'].transform('count')

In [12]:
train.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,...,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio,tipo_techo,flias_afectadas
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,r,n,f,q,t,d,...,0,0,0,0,0,0,3,25.82,9299,1
28830,2,10,8,7,r,n,x,q,s,d,...,0,0,0,0,0,0,2,25.82,19889,1
94947,2,10,5,5,r,n,f,x,t,d,...,0,0,0,0,0,0,3,25.82,19889,1
590882,2,10,6,5,r,n,f,x,s,d,...,0,0,0,0,0,0,2,25.82,19889,1
201944,3,30,8,9,r,n,f,x,s,d,...,0,0,0,0,0,0,3,30.18,9299,1


In [13]:
train = train.drop('roof_type', axis=1)
train = train.drop('other_floor_type', axis=1)
train = train.drop('position', axis=1)
#train = train.drop('other_floor_type', axis=1)
train = train.drop('plan_configuration', axis=1)
train = train.drop('legal_ownership_status', axis=1)
train = train.drop('ground_floor_type', axis=1)
train = train.drop('foundation_type', axis=1)

In [14]:
train_value = train['damage_grade']
train_data = train.iloc[:, train.columns != 'damage_grade']

In [15]:
train_data

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,edad_promedio,tipo_techo,flias_afectadas
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,1,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,9299,1
28830,2,10,8,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
94947,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
590882,2,10,6,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
201944,3,30,8,9,1,0,0,0,0,0,...,0,0,0,0,0,0,0,30.18,9299,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007913,2,60,11,6,0,0,0,0,0,1,...,0,0,0,0,0,0,0,25.82,1809,1
349181,2,0,8,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,13307,1
1009561,1,5,15,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,17.64,17361,1


In [16]:
test_labels = pd.read_csv('test_train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id')
test_values = pd.read_csv('test_train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

In [17]:
test = test_values.merge(test_labels, on='building_id')

In [18]:
test = test.drop('geo_level_1_id', axis=1)
test = test.drop('geo_level_2_id', axis=1)
test = test.drop('geo_level_3_id', axis=1)
test = test.drop('land_surface_condition', axis=1)

In [19]:
#edad promedio segun cantidad de pisos
test['edad_promedio'] = test.groupby('count_floors_pre_eq')['age'].transform('mean')

In [20]:
#cantidad de tipos de techo segun edad
test['tipo_techo'] = test.groupby('age')['roof_type'].transform('count')

In [21]:
#cantidad de familias por edificacion
test['cantidad_flias'] = test.groupby('building_id')['count_families'].transform('count')

In [22]:
test = test.drop('roof_type', axis=1)
test = test.drop('other_floor_type', axis=1)
test = test.drop('position', axis=1)
#test = test.drop('other_floor_type', axis=1)
test = test.drop('plan_configuration', axis=1)
test = test.drop('legal_ownership_status', axis=1)
test = test.drop('ground_floor_type', axis=1)
test = test.drop('foundation_type', axis=1)


#test = test.drop_duplicates('damage_grade')
IDs = test[['damage_grade']].copy()
#test = test.drop('damage_grade', axis=1)

In [23]:
test.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio,tipo_techo,cantidad_flias
building_id,,,,,,,,,,,,,,,,,,,,,
1045849,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,2,26.61,6255,1
523230,2,15,11,5,0,1,0,0,0,0,...,0,0,0,0,0,0,3,26.61,5750,1
183337,4,80,9,10,0,0,0,0,0,1,...,0,0,0,0,0,0,2,59.25,482,1
1038657,3,40,9,7,0,1,0,0,0,0,...,0,0,0,0,0,0,2,30.07,1715,1
237062,2,15,3,4,0,1,0,0,0,0,...,0,0,0,0,0,0,2,26.61,5750,1


In [24]:
test_value = test['damage_grade']
test_data = test.iloc[:, test.columns != 'damage_grade']

In [25]:
test.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio,tipo_techo,cantidad_flias
building_id,,,,,,,,,,,,,,,,,,,,,
1045849,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,2,26.61,6255,1
523230,2,15,11,5,0,1,0,0,0,0,...,0,0,0,0,0,0,3,26.61,5750,1
183337,4,80,9,10,0,0,0,0,0,1,...,0,0,0,0,0,0,2,59.25,482,1
1038657,3,40,9,7,0,1,0,0,0,0,...,0,0,0,0,0,0,2,30.07,1715,1
237062,2,15,3,4,0,1,0,0,0,0,...,0,0,0,0,0,0,2,26.61,5750,1


In [26]:
test_data.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,edad_promedio,tipo_techo,cantidad_flias
building_id,,,,,,,,,,,,,,,,,,,,,
1045849,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,26.61,6255,1
523230,2,15,11,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,26.61,5750,1
183337,4,80,9,10,0,0,0,0,0,1,...,0,0,0,0,0,0,0,59.25,482,1
1038657,3,40,9,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,30.07,1715,1
237062,2,15,3,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,26.61,5750,1


In [27]:
LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=False)

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(train_data, train_value, train_size=0.7, random_state=1234)

In [29]:
train_data

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,edad_promedio,tipo_techo,flias_afectadas
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,1,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,9299,1
28830,2,10,8,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
94947,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
590882,2,10,6,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
201944,3,30,8,9,1,0,0,0,0,0,...,0,0,0,0,0,0,0,30.18,9299,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007913,2,60,11,6,0,0,0,0,0,1,...,0,0,0,0,0,0,0,25.82,1809,1
349181,2,0,8,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,13307,1
1009561,1,5,15,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,17.64,17361,1


In [30]:
categorical_features_indices = np.where(train_data.dtypes == np.object)[0]

In [31]:
%%capture
modelCatBoost=CatBoostRegressor(iterations=250, depth=3, learning_rate=0.05)
modelCatBoost.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_validation, y_validation),plot=True)

0:	learn: 0.6068625	test: 0.6087446	best: 0.6087446 (0)	total: 63.7ms	remaining: 15.9s
1:	learn: 0.6031348	test: 0.6050190	best: 0.6050190 (1)	total: 75.6ms	remaining: 9.37s
2:	learn: 0.5996545	test: 0.6015368	best: 0.6015368 (2)	total: 85.5ms	remaining: 7.04s
3:	learn: 0.5963590	test: 0.5981649	best: 0.5981649 (3)	total: 95.7ms	remaining: 5.89s
4:	learn: 0.5933727	test: 0.5951259	best: 0.5951259 (4)	total: 106ms	remaining: 5.2s
5:	learn: 0.5906645	test: 0.5923670	best: 0.5923670 (5)	total: 117ms	remaining: 4.75s
6:	learn: 0.5881384	test: 0.5897973	best: 0.5897973 (6)	total: 126ms	remaining: 4.38s
7:	learn: 0.5858579	test: 0.5875215	best: 0.5875215 (7)	total: 137ms	remaining: 4.13s
8:	learn: 0.5838111	test: 0.5854740	best: 0.5854740 (8)	total: 146ms	remaining: 3.92s
9:	learn: 0.5818068	test: 0.5834291	best: 0.5834291 (9)	total: 158ms	remaining: 3.8s
10:	learn: 0.5800783	test: 0.5816990	best: 0.5816990 (10)	total: 168ms	remaining: 3.65s
11:	learn: 0.5785273	test: 0.5801190	best: 0.58011

111:	learn: 0.5504347	test: 0.5521479	best: 0.5521479 (111)	total: 1.35s	remaining: 1.66s
112:	learn: 0.5504068	test: 0.5521334	best: 0.5521334 (112)	total: 1.38s	remaining: 1.68s
113:	learn: 0.5503790	test: 0.5521064	best: 0.5521064 (113)	total: 1.39s	remaining: 1.66s
114:	learn: 0.5503477	test: 0.5520876	best: 0.5520876 (114)	total: 1.4s	remaining: 1.65s
115:	learn: 0.5503140	test: 0.5520599	best: 0.5520599 (115)	total: 1.41s	remaining: 1.63s
116:	learn: 0.5502821	test: 0.5520327	best: 0.5520327 (116)	total: 1.42s	remaining: 1.62s
117:	learn: 0.5502549	test: 0.5520128	best: 0.5520128 (117)	total: 1.43s	remaining: 1.6s
118:	learn: 0.5502036	test: 0.5519823	best: 0.5519823 (118)	total: 1.44s	remaining: 1.59s
119:	learn: 0.5501829	test: 0.5519598	best: 0.5519598 (119)	total: 1.45s	remaining: 1.57s
120:	learn: 0.5501393	test: 0.5519213	best: 0.5519213 (120)	total: 1.46s	remaining: 1.56s
121:	learn: 0.5501102	test: 0.5518974	best: 0.5518974 (121)	total: 1.47s	remaining: 1.55s
122:	learn: 

204:	learn: 0.5481436	test: 0.5501500	best: 0.5501500 (204)	total: 2.4s	remaining: 527ms
205:	learn: 0.5481338	test: 0.5501400	best: 0.5501400 (205)	total: 2.41s	remaining: 515ms
206:	learn: 0.5481115	test: 0.5501265	best: 0.5501265 (206)	total: 2.42s	remaining: 503ms
207:	learn: 0.5480795	test: 0.5500974	best: 0.5500974 (207)	total: 2.43s	remaining: 491ms
208:	learn: 0.5480719	test: 0.5500960	best: 0.5500960 (208)	total: 2.44s	remaining: 479ms
209:	learn: 0.5480430	test: 0.5500698	best: 0.5500698 (209)	total: 2.45s	remaining: 467ms
210:	learn: 0.5480387	test: 0.5500697	best: 0.5500697 (210)	total: 2.46s	remaining: 455ms
211:	learn: 0.5480233	test: 0.5500603	best: 0.5500603 (211)	total: 2.47s	remaining: 443ms
212:	learn: 0.5480176	test: 0.5500586	best: 0.5500586 (212)	total: 2.48s	remaining: 431ms
213:	learn: 0.5480050	test: 0.5500525	best: 0.5500525 (213)	total: 2.5s	remaining: 421ms
214:	learn: 0.5479996	test: 0.5500499	best: 0.5500499 (214)	total: 2.52s	remaining: 410ms
215:	learn: 

In [41]:
preds = modelCatBoost.predict(X_train)

logloss = log_loss(y_train, preds)
print("logloss: %f" % (logloss))

ValueError: y_true and y_pred contain different number of classes 3, 2. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [1 2 3]

In [ ]:
pd.to_numeric(preds.round(), downcast='signed')

In [39]:
type(y_train)

pandas.core.series.Series

In [40]:
type(preds)

numpy.ndarray

In [ ]:
%%capture
modelCatBoost.fit(train_data,train_value,cat_features=categorical_features_indices)
preds = modelCatBoost.predict(test)
entregable_CatBoost = IDs[['building_id']].copy()
entregable_CatBoost['damage_grade'] = preds

In [ ]:
entregable_CatBoost.to_csv('CatBoost.csv', index=False)

In [ ]:
modeloCB = CatBoostRegressor()
LogLoss = make_scorer(log_loss, greater_is_better = False)
categorical_features_indices = np.where(train_data.dtypes == np.object)[0]

In [ ]:
parameters = {'iterations':[100, 250, 500] , 'depth':[2, 3, 5], 'learning_rate':[0.005, 0.05, 0.5]}
gscv = GridSearchCV(modeloCB, parameters, cv=5, scoring=LogLoss)

In [ ]:
%%capture
gscv.fit(train_data, train_values, cat_features = categorical_features_indices)

In [ ]:
print(gscv.best_params_)

In [ ]:
%%capture
modeloCB = CatBoostRegressor(depth = 5, iterations = 100, learning_rate = 0.05)
modeloCB.fit(train_data, train_value, cat_features = categorical_features_indices)
result = modeloCB.predict(test)

entregable_CB = IDs[['building_id']].copy() #revisar
entregable_CB['age'] = result #revisar

In [ ]:
entregable_CB.to_csv('CB_Final.csv', index=False)